# ACS Exploratory Data Analysis

In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca
import folium
import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd

pd.set_option('display.max_columns', None) 

import datetime as dt
import time

from calitp import get_engine

engine = get_engine()
connection = engine.connect()

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [3]:
# all CA
query = """SELECT *
FROM `american_community_survey.censustract_2018_5yr` AS acs
WHERE acs.`geo_id` LIKE '06%'"""

acs_ca = pd.read_gbq(query, project_id="cal-itp-data-infra", location="US")

In [4]:
acs_ca >> head (10)

,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_60_to_61,male_62_to_64,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,median_income,income_per_capita,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,pop_determined_poverty_status,poverty,gini_index,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,percent_income_spent_on_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_college,masters_degree,one_year_more_college,less_than_high_school_graduate,high_school_including_ged,bac

In [5]:
# join to boundary file 
# geojson wasn't working, read directly from census ftp
# CA FIPS == 06
tracts_path = "https://www2.census.gov/geo/tiger/TIGER2018/TRACT/tl_2018_06_tract.zip"
tracts_ca = gpd.read_file(tracts_path)

In [22]:
tracts_ca_acs = tracts_ca.merge(acs_ca, how="inner", left_on="GEOID", right_on="geo_id")

In [23]:
tracts_ca_acs >> head(5)

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,geo_id,do_date,total_pop,households,male_pop,female_pop,median_age,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_25_to_29,male_30_to_34,male_35_to_39,male_40_to_44,male_45_to_49,male_50_to_54,male_55_to_59,male_60_to_61,male_62_to_64,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_25_to_29,female_30_to_34,female_35_to_39,female_40_to_44,female_45_to_49,female_50_to_54,female_55_to_59,female_60_to_61,female_62_to_64,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,white_pop,population_1_year_and_over,population_3_years_over,pop_5_years_over,pop_15_and_over,pop_16_over,pop_25_years_over,pop_25_64,pop_never_married,pop_now_married,pop_separated,pop_widowed,pop_divorced,not_us_citizen_pop,black_pop,asian_pop,hispanic_pop,amerindian_pop,other_race_pop,two_or_more_races_pop,hispanic_any_race,not_hispanic_pop,asian_male_45_54,asian_male_55_64,black_male_45_54,black_male_55_64,hispanic_male_45_54,hispanic_male_55_64,white_male_45_54,white_male_55_64,median_income,income_per_capita,income_less_10000,income_10000_14999,income_15000_19999,income_20000_24999,income_25000_29999,income_30000_34999,income_35000_39999,income_40000_44999,income_45000_49999,income_50000_59999,income_60000_74999,income_75000_99999,income_100000_124999,income_125000_149999,income_150000_199999,income_200000_or_more,pop_determined_poverty_status,poverty,gini_index,housing_units,renter_occupied_housing_units_paying_cash_median_gross_rent,owner_occupied_housing_units_lower_value_quartile,owner_occupied_housing_units_median_value,owner_occupied_housing_units_upper_value_quartile,occupied_housing_units,housing_units_renter_occupied,vacant_housing_units,vacant_housing_units_for_rent,vacant_housing_units_for_sale,dwellings_1_units_detached,dwellings_1_units_attached,dwellings_2_units,dwellings_3_to_4_units,dwellings_5_to_9_units,dwellings_10_to_19_units,dwellings_20_to_49_units,dwellings_50_or_more_units,mobile_homes,housing_built_2005_or_later,housing_built_2000_to_2004,housing_built_1939_or_earlier,median_year_structure_built,married_households,nonfamily_households,family_households,households_public_asst_or_food_stamps,male_male_households,female_female_households,children,children_in_single_female_hh,median_rent,percent_income_spent_on_rent,rent_burden_not_computed,rent_over_50_percent,rent_40_to_50_percent,rent_35_to_40_percent,rent_30_to_35_percent,rent_25_to_30_percent,rent_20_to_25_percent,rent_15_to_20_percent,rent_10_to_15_percent,rent_under_10_percent,owner_occupied_housing_units,million_dollar_housing_units,mortgaged_housing_units,different_house_year_ago_different_city,different_house_year_ago_same_city,families_with_young_children,two_parent_families_with_young_children,two_parents_in_labor_force_families_with_young_children,two_parents_father_in_labor_force_families_with_young_children,two_parents_mother_in_labor_force_families_with_young_children,two_parents_not_in_labor_force_families_with_young_children,one_parent_families_with_young_children,father_one_parent_families_with_young_children,father_in_labor_force_one_parent_families_with_young_children,commute_less_10_mins,commute_10_14_mins,commute_15_19_mins,commute_20_24_mins,commute_25_29_mins,commute_30_34_mins,commute_35_44_mins,commute_60_more_mins,commute_45_59_mins,commuters_16_over,walked_to_work,worked_at_home,no_car,no_cars,one_car,two_cars,three_cars,four_more_cars,aggregate_travel_time_to_work,commuters_by_public_transportation,commuters_by_bus,commuters_by_car_truck_van,commuters_by_carpool,commuters_by_subway_or_elevated,commuters_drove_alone,group_quarters,associates_degree,bachelors_degree,high_school_diploma,less_one_year_colle

## Variables of interest:
- total_pop -> denominator; derive pop density
- households -> denominator
- not_us_citizen_pop, black_pop, hispanic_pop
- income_less_10000--income_20000_24999 -> 2018 extremely low income
- income_25000_29999--income_40000_44999 -> 2018 very low income
- income_45000_49999--income_74999 -> 2018 low income
- pop_determined_poverty_status
- poverty (drop this or income cuts)
- no_car, no_cars (one of these must be workers and one must be households, pick one)
- age male


In [24]:
tracts_ca_acs = (tracts_ca_acs
             >> mutate(inc_extremelylow = _.income_less_10000+_.income_10000_14999+_.income_15000_19999+_.income_20000_24999,
                       inc_verylow = _.income_25000_29999+_.income_30000_34999+_.income_35000_39999+_.income_40000_44999,
                       inc_low = _.income_45000_49999+_.income_50000_59999+_.income_60000_74999
                      )
                )

In [25]:
tracts_ca_acs = (tracts_ca_acs           
    >> select("ALAND","geometry","geo_id","total_pop","households", "not_us_citizen_pop","black_pop","hispanic_pop",
                       "inc_extremelylow","inc_verylow","inc_low","pop_determined_poverty_status","poverty",
                       "no_car", "no_cars",
                       _["male_under_5":"male_22_to_24"], _["male_65_to_66":"male_85_and_over"],
                        _["female_under_5":"female_22_to_24"], _["female_65_to_66":"female_85_and_over"]
                      )
            )

tracts_ca_acs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 8057 entries, 0 to 8056
Data columns (total 43 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   ALAND                          8057 non-null   int64   
 1   geometry                       8057 non-null   geometry
 2   geo_id                         8057 non-null   object  
 3   total_pop                      8057 non-null   float64 
 4   households                     8057 non-null   float64 
 5   not_us_citizen_pop             8057 non-null   float64 
 6   black_pop                      8057 non-null   float64 
 7   hispanic_pop                   8057 non-null   float64 
 8   inc_extremelylow               8057 non-null   float64 
 9   inc_verylow                    8057 non-null   float64 
 10  inc_low                        8057 non-null   float64 
 11  pop_determined_poverty_status  8057 non-null   float64 
 12  poverty                   

In [26]:
tracts_ca_acs['male_youth'] = tracts_ca_acs.loc[:,"male_under_5":"male_22_to_24"].sum(axis=1)
tracts_ca_acs['female_youth'] = tracts_ca_acs.loc[:,"female_under_5":"female_22_to_24"].sum(axis=1)

tracts_ca_acs['male_seniors'] = tracts_ca_acs.loc[:,"male_65_to_66":"male_85_and_over"].sum(axis=1)
tracts_ca_acs['female_seniors'] = tracts_ca_acs.loc[:,"female_65_to_66":"female_85_and_over"].sum(axis=1)

In [27]:
tracts_ca_acs >> head (3)

,ALAND,geometry,geo_id,total_pop,households,not_us_citizen_pop,black_pop,hispanic_pop,inc_extremelylow,inc_verylow,inc_low,pop_determined_poverty_status,poverty,no_car,no_cars,male_under_5,male_5_to_9,male_10_to_14,male_15_to_17,male_18_to_19,male_20,male_21,male_22_to_24,male_65_to_66,male_67_to_69,male_70_to_74,male_75_to_79,male_80_to_84,male_85_and_over,female_under_5,female_5_to_9,female_10_to_14,female_15_to_17,female_18_to_19,female_20,female_21,female_22_to_24,female_65_to_66,female_67_to_69,female_70_to_74,female_75_to_79,female_80_to_84,female_85_and_over,male_youth,female_youth,male_seniors,female_seniors
0,3837562,"POLYGON ((-118.58119 34.14318, -118.58099 34.1...",06037137504,2073.0,694.0,23.0,19.0,64.0,30.0,29.0,102.0,2073.0,90.0,12.0,10.0,39.0,46.0,74.0,36.0,30.0,0.0,14.0,27.0,58.0,34.0,70.0,32.0,17.0,55.0,66.0,34.0,46.0,33.0,10.0,0.0,0.0,55.0,46.0,52.0,96.0,24.0,73.0,6.0,266.0,244.0,266.0,297.0
1,4472196,"POLYGON ((-118.60573 34.14585, -118.60561 34.1...",06037138000,4673.0,1784.0,198.0,325.0,393.0,270.0,124.0,196.0,4673.0,386.0,0.0,19.0,159.0,134.0,123.0,55.0,34.0,34.0,0.0,59.0,92.0,126.0,137.0,110.0,17.0,67.0,58.0,99.0,105.0,46.0,41.0,0.0,42.0,34.0,73.0,97.0,115.0,113.0,0.0,89.0,598.0,425.0,549.0,487.0
2,1152031,"POLYGON ((-118.53082 34.18024, -118.52952 34.1...",06037139200,5840.0,2172.0,815.0,153.0,1330.0,242.0,315.0,548.0,5840.0,602.0,66.0,108.0,124.0,217.0,108.0,68.0,116.0,81.0,17.0,46.0,16.0,64.0,73.0,129.0,13.0,14.0,262.0,16.0,224.0,109.0,0.0,0.0,11.0,12.0,97.0,106.0,213.0,27.0,122.0,97.0,777.0,634.0,309.0,662.0


In [28]:
tracts_ca_acs = (tracts_ca_acs
                 >> mutate(youth_pop = _.male_youth+_.female_youth,
                           seniors_pop = _.male_seniors+_.female_seniors
                          )
                 >> select (-_["male_under_5":"male_22_to_24"], -_["male_65_to_66":"male_85_and_over"],
                        -_["female_under_5":"female_22_to_24"], -_["female_65_to_66":"female_85_and_over"]
                           )
                )

tracts_ca_acs.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 8057 entries, 0 to 8056
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   ALAND                          8057 non-null   int64   
 1   geometry                       8057 non-null   geometry
 2   geo_id                         8057 non-null   object  
 3   total_pop                      8057 non-null   float64 
 4   households                     8057 non-null   float64 
 5   not_us_citizen_pop             8057 non-null   float64 
 6   black_pop                      8057 non-null   float64 
 7   hispanic_pop                   8057 non-null   float64 
 8   inc_extremelylow               8057 non-null   float64 
 9   inc_verylow                    8057 non-null   float64 
 10  inc_low                        8057 non-null   float64 
 11  pop_determined_poverty_status  8057 non-null   float64 
 12  poverty                   

In [29]:
shared_utils.utils.geoparquet_gcs_export(tracts_ca_acs, "gs://calitp-analytics-data/data-analyses/ahsc_grant/", "acs_tbl_ca")